In [51]:
import pandas as pd
import glob
import os
import datetime as dt
import shutil

from datetime import datetime
from google.colab import files

# **Spotify**

Account connected with **slazur83@gmail**

In [52]:
# File path to the JSON files

file_list = glob.glob('/content/drive/MyDrive/Dane z aplikacji/Spotify/slazur83@gmail.com/MyData/StreamingHistory*.json')
df_list = []

for filename in file_list:
    with open(filename, encoding='utf-8') as inputfile:
        df = pd.read_json(inputfile)
        df_list.append(df)

df1 = pd.concat(df_list, ignore_index=True)
df1['Account'] = 'slazur83@gmail.com'

Account connected with **zethar182@gmail**

In [53]:
# File path to the JSON files

file_list = glob.glob('/content/drive/MyDrive/Dane z aplikacji/Spotify/zethar182@gmail.com/MyData/StreamingHistory*.json')
df_list = []

for filename in file_list:
    with open(filename, encoding='utf-8') as inputfile:
        df = pd.read_json(inputfile)
        df_list.append(df)

df2 = pd.concat(df_list, ignore_index=True)
df2['Account'] = 'zethar182@gmail.com'

Concatenation of Spotify data

In [54]:
df_spotify = pd.concat([df1,df2], ignore_index=True)
df_spotify['Source'] = 'Spotify'

In [55]:
# Rename columns in the dataframe
df_spotify.rename(columns={'endTime': 'Date', 'artistName': 'Artist', 'trackName': 'Track', 'msPlayed': 'Duration'}, inplace=True)

# Reorder columns in the dataframe
df_spotify = df_spotify[['Artist', 'Track', 'Date', 'Duration', 'Source', 'Account']]

# **LastFM**

In [ ]:
# File name to upload
uploaded_filename = 'slazur83.csv'

# Check if a file with the same name exists and delete it if it exists
if os.path.exists(uploaded_filename):
    creation_time = os.path.getctime(uploaded_filename)
    creation_date = dt.datetime.fromtimestamp(creation_time).strftime('%Y-%m-%d %H:%M:%S')

    os.remove(uploaded_filename)
    print(f"Deleted the existing file {uploaded_filename} created on {creation_date}")

# Upload a new version of the file
uploaded = files.upload()

lastfm_local_path = uploaded_filename

In [57]:
import shutil

lastfm = f'/content/drive/MyDrive/Dane z aplikacji/LastFM/{uploaded_filename}.csv'
shutil.copy(lastfm_local_path, lastfm)

print(f"The file {uploaded_filename} has been moved to appropriate folder in Google Drive.")

The file slazur83.csv has been moved to appropriate folder in Google Drive.


In [59]:
# Read the CSV file into a dataframe
df_lastfm = pd.read_csv(lastfm, header=None, names=['Artist', 'Album', 'Track', 'Date'])

def convert_to_datetime(date_str):
    return datetime.strptime(date_str, "%d %b %Y %H:%M")

df_lastfm['Date'] = df_lastfm['Date'].apply(convert_to_datetime)

# Add new columns to the dataframe
df_lastfm['Source'] = 'LastFM'
df_lastfm['Account'] = 'slazur83@gmail.com'

# **Deezer**

In [60]:
# File path to the Excel file
deezer = '/content/drive/MyDrive/Dane z aplikacji/Deezer/4519420622.xlsx'

# Read the specified sheet from the Excel file into a dataframe
df_deezer = pd.read_excel(deezer, sheet_name="10_listeningHistory")

# Rename columns in the dataframe
df_deezer.rename(columns={'Song Title': 'Track', 'Album Title': 'Album', 'Listening Time': 'Duration', 'Platform Name':'Platform'}, inplace=True)

# Delete a column 'ISRC' from the dataframe
del df_deezer['ISRC']

# Add new columns to the dataframe
df_deezer['Source'] = 'Deezer'
df_deezer['Account'] = 'slazur83@gmail.com'

# **Concatenatation of the three dataframes into one**

In [ ]:
# Concatenate the three dataframes into one, ignoring their original indices
merged = pd.concat([df_spotify, df_lastfm, df_deezer], ignore_index=True)

# Check if the length of each of the original dataframes df1, df2, and df3 added together is equal to the length of the merged dataframe.
len_df_spotify = sum([len(df_spotify)])
len_df_lastfm = sum([len(df_lastfm)])
len_df_deezer = sum([len(df_deezer)])

if len_df_spotify + len_df_lastfm + len_df_deezer == len(merged):
  print('All data is merged')
else:
  print('Not all data is merged')

# Check if there are nulls in the merged dataframe
nulls = merged['Date'].isna().sum()
if nulls:
  if nulls < 15:
    merged.dropna(subset=['Date'], inplace=True)
    print(f'{nulls} rows with empty Date column have been dropped.')
  else:
    print(f'There are {nulls} nulls. It should be checked')
else:
  print('There are no nulls')

In [ ]:
# Reorder columns in the merged dataframe
merged = merged[['Date', 'Artist', 'Track', 'Album', 'Duration', 'Source', 'Account', 'Platform', 'Platform Model', 'IP Address']]

# Change Date field to time format
merged['Date'] = pd.to_datetime(merged['Date'])
merged.sort_values(by='Date')

In [ ]:
print(f'Length of df_spotify is {len_df_spotify}, lenght of df_lastfm is {len_df_lastfm}, length of df_deezer is {len_df_deezer}')

In [ ]:
print(f"Spotify --> {df_spotify['Date'].min()} to {df_spotify['Date'].max()}\n")
print(f"Deezer --> {df_deezer['Date'].min()} to {df_deezer['Date'].max()}\n")
print(f"LastFM --> since {df_lastfm['Date'].min()} to {df_lastfm['Date'].max()}\n")

In [69]:
# Export the merged dataframe to the music_tracks.csv file
merged.to_csv('/content/drive/MyDrive/Colab Notebooks/Tableau/Outputs/music_tracks.csv', index=False)

In [ ]:
# Import the exported CSV file
try:
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Tableau/Outputs/music_tracks.csv')
    print("CSV file imported successfully.")
except FileNotFoundError:
    print("CSV file not found.")
    df = None